In [4]:
# Return time cocnepts of a certain level inside a vocabulary scheme  
# Here are two exmaples: one for the North American vocabulary, the other for 
# the international vocabulary 

install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)

Loading required package: XML

Loading required package: RCurl



In [5]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
prefix dc: <http://purl.org/dc/elements/1.1/> 
prefix dcterms: <http://purl.org/dc/terms/> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix geo: <http://www.opengis.net/ont/geosparql#> 
prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#> 
prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rank: <http://resource.geosciml.org/ontology/timescale/rank/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix samfl: <http://def.seegrid.csiro.au/ontology/om/sam-lite#> 
prefix sf: <http://www.opengis.net/ont/sf#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix sosa: <http://www.w3.org/ns/sosa/> 
prefix thors: <http://resource.geosciml.org/ontology/timescale/thors#> 
prefix time: <http://www.w3.org/2006/time#> 
prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
prefix vann: <http://purl.org/vocab/vann/> 
prefix void: <http://rdfs.org/ns/void#> 
prefix xkos: <http://rdf-vocabulary.ddialliance.org/xkos#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix tsna: <http://deeptimekb.org/tsna#> 
prefix tssc: <http://deeptimekb.org/tssc#> 
prefix tsnc: <http://deeptimekb.org/tsnc#> 
prefix tsnz: <http://deeptimekb.org/tsnz#> 
prefix tswe: <http://deeptimekb.org/tswe#> 
prefix tsbr: <http://deeptimekb.org/tsbr#> 
prefix tsba: <http://deeptimekb.org/tsba#> 
prefix tsjp: <http://deeptimekb.org/tsjp#> 
prefix tsau: <http://deeptimekb.org/tsau#> 
        "

In [21]:
ConceptLevels <- c("Age", "Epoch", "Sub-Period", "Period", "Era", "Eon", "Super-Eon")
cLevelLength = length(ConceptLevels)

cat("===Available time concept levels:", "\n" )
for(k in 1:cLevelLength) 
    {
      print(ConceptLevels[k])
    }

# Get a list of vocabulary scheme URIs

q0 = paste0(
              sparql_prefix, 
                  '
                   SELECT   DISTINCT  ?schURI  str(?lbl) AS ?schemeName
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                             rdfs:label ?lbl .
                     }      
                   }
                  '
              )

# run the query  
schemeList = SPARQL(endpoint, q0)$results
  
schListLength =length(schemeList$schURI)

cat("\n","===Available vocabulary scheme IDs:", "\n" )
for(k in 1:schListLength) 
    {
       cleanStr<-substr(schemeList$schURI[k], 9, nchar(schemeList$schURI[k])-1)
       schID <- strsplit(as.character(cleanStr), "/")
       n <- length(schID[[1]]) #use n to locate the last item in the splitted scheme URI, note the split result is like a matrix  
       print (schID[[1]][n]) #use this structure schID[[1]][n] to find the last item in the split result, e.g. 'tsnc2019' 
    }


===Available time concept levels: 
[1] "Age"
[1] "Epoch"
[1] "Sub-Period"
[1] "Period"
[1] "Era"
[1] "Eon"
[1] "Super-Eon"

 ===Available vocabulary scheme IDs: 
[1] "tsnc2019"
[1] "tssc2019"
[1] "tsnz2019"
[1] "tsau2019"
[1] "tsba2019"
[1] "tsbr2019"
[1] "tsjp2019"
[1] "tswe2019"
[1] "tsna2019"
[1] "isc2004-04"
[1] "isc2005-12"
[1] "isc2006-04"
[1] "isc2008-08"
[1] "isc2009-08"
[1] "isc2010-09"
[1] "isc2012-08"
[1] "isc2013-01"
[1] "isc2014-02"
[1] "isc2014-10"
[1] "isc2015-01"
[1] "isc2016-04"
[1] "isc2016-10"
[1] "isc2017-02"
[1] "isc2018-07"
[1] "isc2018-08"


In [35]:
userConceptLevel <- readline(prompt="Enter a concept level from the above list: ")
userSchemeID <- readline(prompt="Enter a vocabulary scheme ID from the above list: ")

Enter a concept level from the above list: Epoch
Enter a vocabulary scheme ID from the above list: tsna2019


In [36]:
# the query code
userConceptLevel1 <- paste("gts:", userConceptLevel, sep="")
userSchemeID1 <- paste("ts:", userSchemeID, sep="")

cat("\n")
cat("===Selected concept level:", userConceptLevel1, "\n")
cat("===Selected scheme ID:", userSchemeID1, "\n")  

# In the sparql code below we use a UNION clause to tackel the ontology patterns for 
#  version control of the international vocabulary schemes 
    
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT  str(?lbl) AS ?conceptName ?baseNum
WHERE
{
                
   GRAPH <http://deeptimekb.org/iscallnew>
   { 

     {
      ?concept  a gts:GeochronologicEra ,
                 ', userConceptLevel1, ' ; 
               skos:inScheme ', userSchemeID1 , '; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition . 
     ?basePosition time:inTemporalPosition ?baseTime .
     ?baseTime time:numericPosition ?baseNum .
    }
    UNION 
    { 
       ?concept  a gts:GeochronologicEra ,
                 ', userConceptLevel1, ' ; 
               skos:inScheme ', userSchemeID1 , '; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ; 
                       skos:inScheme ', userSchemeID1 , '
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ', userSchemeID1 , '
                   ] .
     }     

    
  }
   
}
ORDER BY ?baseNum
                  '
              )
# run the query  
resConcepts = SPARQL(endpoint, q)$results
  
print(resConcepts)


===Selected concept level: gts:Epoch 
===Selected scheme ID: ts:tsna2019 
          conceptName baseNum
1       Gulfian Epoch  100.50
2    Comanchean Epoch  128.09
3     Coahulian Epoch  145.01
4        Ochoan Epoch  259.81
5   Guadalupian Epoch  272.30
6    Leonardian Epoch  284.02
7   Wolfcampian Epoch  298.88
8  Cincinnatian Epoch  450.96
9     Mohawkian Epoch  456.72
10      Ibexian Epoch  486.73
11    Millardan Epoch  497.00
12   Lincolnian Epoch  511.40
13     Waucoban Epoch  519.25
14     Begadean Epoch  541.00
